In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


In [5]:
df=pd.read_csv("/kaggle/input/playground-series-s5e7/train.csv")

In [6]:
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert


In [7]:
df.isnull().sum()

id                              0
Time_spent_Alone             1190
Stage_fear                   1893
Social_event_attendance      1180
Going_outside                1466
Drained_after_socializing    1149
Friends_circle_size          1054
Post_frequency               1264
Personality                     0
dtype: int64

In [8]:
df.Time_spent_Alone.describe()

count    17334.000000
mean         3.137764
std          3.003786
min          0.000000
25%          1.000000
50%          2.000000
75%          4.000000
max         11.000000
Name: Time_spent_Alone, dtype: float64

## data preprocessing

1. for Time_spent_Alone we replace with *MEDIAN*
2. for Stage_fear we replace with *MODE*
3. for Social_event_attendance & Going_outside we replace with *MEAN*
4. for Drained_after_socializing we replace with *MODE*
5. for Friends_circle_size, Post_frequency we replace *MEDIAN*

* NOW WE FOUND A WAY TO MANAGE MISSING VALUES..! LET US DEFINE A FUNCTION TO PREPROCESS THE DATA ACCORDING TO OUR ASSUMPTIONS AND STATS




## 1. data encoding
1. encoded stage fear, drained after socializing into 0 and 1 from No, Yes
2. encoded personality like
    * if extrovert then 0
    * if introvert then 1

In [13]:
def data_preprocess(data):
    df=data.copy()
    df.Time_spent_Alone.fillna(df.Time_spent_Alone.median(),inplace=True)
    df.Stage_fear.fillna(df.Stage_fear.mode()[0],inplace=True)
    df.Social_event_attendance.fillna(df.Social_event_attendance.mean(),inplace=True)
    df.Going_outside.fillna(df.Going_outside.mean(),inplace=True)
    df.Drained_after_socializing.fillna(df.Drained_after_socializing.mode()[0],inplace=True)
    df.Friends_circle_size.fillna(df.Friends_circle_size.median(),inplace=True)
    df.Post_frequency.fillna(df.Post_frequency.median(),inplace=True)
    df.Stage_fear=df.Stage_fear.map({'Yes':1,'No':0})
    df.Drained_after_socializing=df.Drained_after_socializing.map({'Yes':1,'No':0})

    return df

In [14]:
df=data_preprocess(df)

/tmp/ipykernel_36/816283456.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.Time_spent_Alone.fillna(df.Time_spent_Alone.median(),inplace=True)
/tmp/ipykernel_36/816283456.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpla

In [15]:
df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,0,6.0,4.0,0,15.0,5.0,Extrovert
1,1,1.0,0,7.0,3.0,0,10.0,8.0,Extrovert
2,2,6.0,1,1.0,0.0,0,3.0,0.0,Introvert
3,3,3.0,0,7.0,3.0,0,11.0,5.0,Extrovert
4,4,1.0,0,4.0,4.0,0,13.0,5.0,Extrovert


In [16]:
df.Personality=df.Personality.map({'Extrovert':0,'Introvert':1})

In [17]:
df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,0,6.0,4.0,0,15.0,5.0,0
1,1,1.0,0,7.0,3.0,0,10.0,8.0,0
2,2,6.0,1,1.0,0.0,0,3.0,0.0,1
3,3,3.0,0,7.0,3.0,0,11.0,5.0,0
4,4,1.0,0,4.0,4.0,0,13.0,5.0,0


In [26]:
X_train=df.drop("Personality",axis=1)
y_train=df.Personality

NOW WE HAVE CLEAN DATA. NOW ITS TIME TO MODEL SELECTION. LET US USE DIFFERNT MODELS IN CLASSIFICATION
* RANDOM FOREST CLASSIFER

In [20]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()

In [21]:
from sklearn.model_selection import RandomizedSearchCV

In [23]:
param_grid = {
    "n_estimators": [100, 200, 300],
    "criterion": ["gini", "entropy", "log_loss"], # 'mse' is deprecated, use 'squared_error'
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": [1.0, 'sqrt', 'log2',None] # 1.0 is equivalent to n_features for regression
}

In [24]:
random_search = RandomizedSearchCV(
    estimator=rfc,
    param_distributions=param_grid, # Can still use a dict of lists like GridSearchCV
    n_iter=50, # Try 50 random combinations, for example
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    random_state=42 # Important for reproducibility
)

In [27]:
random_search.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END criterion=log_loss, max_depth=10, max_features=1.0, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=  13.8s
[CV] END criterion=gini, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   5.0s
[CV] END criterion=log_loss, max_depth=10, max_features=None, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=   5.3s
[CV] END criterion=log_loss, max_depth=10, max_features=None, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=   4.6s
[CV] END criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=300; total time=   4.9s
[CV] END criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=300; total time=   5.0s
[CV] END criterion=entropy, max_depth=20, max_features=None, min_samples_leaf=4, min_samples_split=5

RandomizedSearchCV(estimator=RandomForestClassifier(), n_iter=50, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [None, 10, 20],
                                        'max_features': [1.0, 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [28]:
model=random_search.best_estimator_

In [29]:
test_data=pd.read_csv("/kaggle/input/playground-series-s5e7/test.csv")

In [31]:
X_test=data_preprocess(test_data)

/tmp/ipykernel_36/816283456.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.Time_spent_Alone.fillna(df.Time_spent_Alone.median(),inplace=True)
/tmp/ipykernel_36/816283456.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpla

In [32]:
y_pred=model.predict(X_test)

In [35]:
prediction=pd.DataFrame(y_pred)

In [36]:
prediction

,0
0,0
1,1
2,0
3,0
4,1
...,...
6170,0
6171,1
6172,0
6173,0


In [38]:
prediction[0]=prediction[0].apply(lambda x:'Introvert' if x == 1 else 'Extrovert' )

In [39]:
prediction

,0
0,Extrovert
1,Introvert
2,Extrovert
3,Extrovert
4,Introvert
...,...
6170,Extrovert
6171,Introvert
6172,Extrovert
6173,Extrovert


In [ ]:
# df = pd.DataFrame(
#     np.random.rand(100, 5),
#     columns=['a', 'b', 'c', 'd', 'e'])
# df.to_csv('/kaggle/working/df.csv',index=False)

In [41]:
submission_df = pd.DataFrame({
    "id": test_data.id,
    "Personality" : prediction[0]
})

In [42]:
submission_df

,id,Personality
0,18524,Extrovert
1,18525,Introvert
2,18526,Extrovert
3,18527,Extrovert
4,18528,Introvert
...,...,...
6170,24694,Extrovert
6171,24695,Introvert
6172,24696,Extrovert
6173,24697,Extrovert


In [43]:
submission_df.to_csv("submission.csv",index=False)